## Training on PPU-6 Dataset

In [4]:
import os
import sys
import tensorflow.compat.v1 as tf
import cv2
import os
import sys

imsize = 640

# Clone efficientdet repo - DO THIS AT THE BEGGINING AND RESTART RUNTIME WITH THE PROVIDED BUTTON
!mkdir /content/drive/MyDrive/training-effdet/
%cd /content/drive/MyDrive/training-effdet/

if "efficientdet" not in os.getcwd():
    !git clone --depth 1 https://github.com/google/automl
    os.chdir('automl/efficientdet')
    sys.path.append('.')
    !pip install -r requirements.txt
    !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
else:
    !git pull

mkdir: cannot create directory ‘/content/drive/MyDrive/training-effdet/’: File exists
/content/drive/MyDrive/training-effdet
fatal: destination path 'automl' already exists and is not an empty directory.
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-80l1_g0s
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-80l1_g0s
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263920 sha256=643e4613679ad796fafac2342bafdb4a1a1e9bda00a023e7b7975de1d15f5223
  Stored in directory: /tmp/pip-ephem-wheel-cache-okxdoh4g/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-3t5g6jun
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-3t5g6jun
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263917

### Prepare data

In [11]:
!mkdir /content/raw-data/
%cd /content/raw-data/

# download data, uncompress & clear the compressed file from disk
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1D-oBYlsD2c4dWnMyhtav1_mYnqfNK-ep' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1D-oBYlsD2c4dWnMyhtav1_mYnqfNK-ep" -O PPU-6.rar && rm -rf /tmp/cookies.txt
!unrar x -y PPU-6.rar

# Create one text file per split that point to each of the images in it
for split in ['train', 'valid', 'test']:
    with open(f'/content/raw-data/{split}.txt', 'w') as file:
        dir = os.listdir(f'/content/raw-data/{split}')
        image_list = [image for image in dir if image[-1]!='t']
        for image in image_list:
            file.write(f'/content/raw-data/{split}/' + image +'\n')


/content/raw-data
--2021-03-28 11:51:23--  https://docs.google.com/uc?export=download&confirm=Ruef&id=1D-oBYlsD2c4dWnMyhtav1_mYnqfNK-ep
Resolving docs.google.com (docs.google.com)... 108.177.98.139, 108.177.98.102, 108.177.98.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.98.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-64-docs.googleusercontent.com/docs/securesc/47l68vfp38mddcppi8aaufbi6ms7lh2a/s6odcotn8lus165l6um4n4rco1o40dke/1616932275000/08364207993842942035/14616866303590944986Z/1D-oBYlsD2c4dWnMyhtav1_mYnqfNK-ep?e=download [following]
--2021-03-28 11:51:23--  https://doc-04-64-docs.googleusercontent.com/docs/securesc/47l68vfp38mddcppi8aaufbi6ms7lh2a/s6odcotn8lus165l6um4n4rco1o40dke/1616932275000/08364207993842942035/14616866303590944986Z/1D-oBYlsD2c4dWnMyhtav1_mYnqfNK-ep?e=download
Resolving doc-04-64-docs.googleusercontent.com (doc-04-64-docs.googleusercontent.com)... 173.194.202.132, 2607:f8b0:400e

In [ ]:
%cd /content/raw-data/
# Resizing images to network size
for split in ['train', 'valid', 'test']:
    with open(f'/content/raw-data/{split}.txt') as file:
        all_pics = file.read().splitlines() 
        total = len(all_pics)
        for idx, name in enumerate(all_pics):
            pic = cv2.imread(name) 
            pic = cv2.resize(pic, (imsize,imsize))
            cv2.imwrite(name, pic)
            print(f'\rConverting {split} split... {idx+1} / {total} converted. Current size {pic.shape}', end='')

/content/raw-data
Converting test split... 520 / 520 converted. Current size (640, 640, 3)

In [ ]:
# create a obj.names file to hold the names of the classes

names = '''\
backpack
helmet
drill
extinguisher
survivor
rope'''

%cd /content/raw-data/
with open('obj.names','w') as file:
    file.write(names)

/content/raw-data


In [ ]:
# Download yolo2tfrecord. This is a modified file to match some tf sublibraries
# Find source file at https://github.com/AlessioTonioni/tf-objdetector
# CHANGE LINE 115/116 depending on the image extension lenght!! (.jpg vs .jpeg)

%cd /content/raw-data/
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CE3-DxC_BsAxRdjO4v5-87ury17FhA_K' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CE3-DxC_BsAxRdjO4v5-87ury17FhA_K" -O yolo2tfrecord_fixes.py && rm -rf /tmp/cookies.txt


/content/raw-data
--2021-03-25 23:31:19--  https://docs.google.com/uc?export=download&confirm=&id=1CE3-DxC_BsAxRdjO4v5-87ury17FhA_K
Resolving docs.google.com (docs.google.com)... 74.125.195.113, 74.125.195.138, 74.125.195.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-80-docs.googleusercontent.com/docs/securesc/d2lcegj6hhnogccp7acf1sve2cr6rf1o/nan7ol1up9c2n8ngmf6sema3qs6dbb15/1616715075000/08364207993842942035/02101882668172567094Z/1CE3-DxC_BsAxRdjO4v5-87ury17FhA_K?e=download [following]
--2021-03-25 23:31:19--  https://doc-0k-80-docs.googleusercontent.com/docs/securesc/d2lcegj6hhnogccp7acf1sve2cr6rf1o/nan7ol1up9c2n8ngmf6sema3qs6dbb15/1616715075000/08364207993842942035/02101882668172567094Z/1CE3-DxC_BsAxRdjO4v5-87ury17FhA_K?e=download
Resolving doc-0k-80-docs.googleusercontent.com (doc-0k-80-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08:

In [ ]:
# Generate the tfrecords for each split & save them to drive

%cd /content/raw-data/

!python /content/raw-data/yolo2tfrecord_fixes.py -t /content/raw-data/train.txt -o /content/PPU-6-{imsize}-train.tfrecord -c /content/raw-data/obj.names
!python /content/raw-data/yolo2tfrecord_fixes.py -t /content/raw-data/valid.txt -o /content/PPU-6-{imsize}-valid.tfrecord -c /content/raw-data/obj.names
!python /content/raw-data/yolo2tfrecord_fixes.py -t /content/raw-data/test.txt -o /content/PPU-6-{imsize}-test.tfrecord -c /content/raw-data/obj.names

!cp /content/PPU-6-{imsize}-train.tfrecord /content/drive/MyDrive/TFM/training-assets
!cp /content/PPU-6-{imsize}-valid.tfrecord /content/drive/MyDrive/TFM/training-assets
!cp /content/PPU-6-{imsize}-test.tfrecord /content/drive/MyDrive/TFM/training-assets

/content/raw-data
2021-03-25 23:31:33.031525: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
911/912
TFRecord saved, creating label_name.pbtxt
Conversion Done
2021-03-25 23:31:37.408503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
274/275
TFRecord saved, creating label_name.pbtxt
Conversion Done
2021-03-25 23:31:40.239516: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
519/520
TFRecord saved, creating label_name.pbtxt
Conversion Done


In [ ]:
# check that the tfrecord has the appropiate info
for i, example in enumerate(tf.python_io.tf_record_iterator("/content/drive/MyDrive/TFM/training-assets/PPU-6-512-train.tfrecord")):
    pass
print(str(i+1) + ' items in train tfrecord')

for i, example in enumerate(tf.python_io.tf_record_iterator("/content/drive/MyDrive/TFM/training-assets/PPU-6-512-valid.tfrecord")):
    pass
print(str(i+1) + ' items in valid tfrecord')

for i, example in enumerate(tf.python_io.tf_record_iterator("/content/drive/MyDrive/TFM/training-assets/PPU-6-512-test.tfrecord")):
    pass
print(str(i+1) + ' items in test tfrecord')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
912 items in train tfrecord
275 items in valid tfrecord
520 items in test tfrecord


In [ ]:
# Create a yaml file with some net properties - MIGHT EXIST ALREADY

with open('/content/raw-data/obj.names','r') as file:
    names = [str(i) + ': ' + name.replace('\n',', ') for i, name in enumerate(file.readlines())]

filecontent = f'''\
num_classes: {len(names)}  
var_freeze_expr: '(efficientnet|fpn_cells|resample_p6)'
label_map: {"{" + "".join(names) + "}"}
moving_average_decay: 0'''
 
with open('/content/PPU-6-config.yaml','w') as file:
    file.write(filecontent)

!cp /content/PPU-6-config.yaml /content/drive/MyDrive/TFM/training-assets

### Start network 

In [ ]:
train_pattern = f'PPU-6-{imsize}-train.tfrecord'
train_images_per_epoch = 912 // 64 * 64             # round to batchsize

valid_pattern = f'PPU-6-{imsize}-valid.tfrecord' 
valid_images_per_epoch = 275 // 8 * 8               # round to batchsize

print('Train images_per_epoch = {}'.format(train_images_per_epoch))
print('Valid images_per_epoch = {}'.format(valid_images_per_epoch))

Train images_per_epoch = 896
Valid images_per_epoch = 272


In [ ]:
# Train the efficientdet
%cd /content/drive/MyDrive/training-effdet/automl/efficientdet

# Touch the logfile to ensure it exists & same for model dir
!touch pablos-logfile-effdet-d1-train.txt
!mkdir /content/drive/MyDrive/training-effdet/PPU-6-640-effdet-d1

# Download the model if neccessary
MODEL = 'efficientdet-d1'
if MODEL not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
    !tar xf {MODEL}.tar.gz

# Train!
!python main.py \
    --mode=train_and_eval \
    --train_file_pattern=/content/drive/MyDrive/TFM/training-assets/{train_pattern} \
    --val_file_pattern=/content/drive/MyDrive/TFM/training-assets/{valid_pattern} \
    --model_name={MODEL} \
    --model_dir=/content/drive/MyDrive/training-effdet/PPU-6-640-effdet-d1 \
    --ckpt={MODEL} \
    --train_batch_size=64 \
    --eval_batch_size=8 \
    --eval_samples={valid_images_per_epoch} \
    --num_examples_per_epoch={train_images_per_epoch} \
    --num_epochs=99999999  \
    --hparams=/content/drive/MyDrive/TFM/training-assets/PPU-6-config.yaml \
    --run_epoch_in_child_process=True \
    >> pablos-logfile-effdet-d1-train.txt

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:Saving checkpoints for 1358 into /content/drive/MyDrive/training-effdet/PPU-6-640-effdet-d1/model.ckpt.
I0326 02:55:20.201018 140551769544576 basic_session_run_hooks.py:618] Saving checkpoints for 1358 into /content/drive/MyDrive/training-effdet/PPU-6-640-effdet-d1/model.ckpt.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
W0326 02:55:20.329272 140551769544576 deprecation.py:339] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/training/saver.py:970: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1358...
I0326 02:55:21.547652 140551769544576 basic_session_run_hooks.py:626] Calling checkpoint listeners after saving checkpoint 1358...
INFO:te

In [ ]:
# Resume training from an existing checkpoint
%cd /content/drive/MyDrive/training-effdet/automl/efficientdet

MODEL = 'efficientdet-d1'
if MODEL not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
    !tar xf {MODEL}.tar.gz

!python main.py \
    --mode=train_and_eval \
    --train_file_pattern=/content/drive/MyDrive/TFM/training-assets/{train_pattern} \
    --val_file_pattern=/content/drive/MyDrive/TFM/training-assets/{valid_pattern} \
    --model_name={MODEL} \
    --model_dir=/content/drive/MyDrive/training-effdet/PPU-6-640-effdet-d1 \
    --ckpt=/content/drive/MyDrive/training-effdet/PPU-6-640-effdet-d1 \
    --train_batch_size=64 \
    --eval_batch_size=8 \
    --eval_samples={valid_images_per_epoch} \
    --num_examples_per_epoch={train_images_per_epoch} \
    --num_epochs=99999999  \
    --hparams=/content/drive/MyDrive/TFM/training-assets/PPU-6-config.yaml \
    --run_epoch_in_child_process=True \
    >> pablos-logfile-effdet-d1-train.txt

Streaming output truncated to the last 5000 lines.
I0328 11:18:29.250066 140037153568640 efficientnet_model.py:374] Block blocks_5 input shape: (64, 160, 160, 24)
I0328 11:18:29.281491 140037153568640 efficientnet_model.py:390] Expand shape: (64, 160, 160, 144)
I0328 11:18:29.311638 140037153568640 efficientnet_model.py:393] DWConv shape: (64, 80, 80, 144)
I0328 11:18:29.334937 140037153568640 efficientnet_model.py:195] Built SE se : (64, 1, 1, 144)
I0328 11:18:29.369967 140037153568640 efficientnet_model.py:414] Project shape: (64, 80, 80, 40)
I0328 11:18:29.370325 140037153568640 efficientnet_model.py:756] block_6 survival_prob: 0.9478260869565217
I0328 11:18:29.370688 140037153568640 efficientnet_model.py:374] Block blocks_6 input shape: (64, 80, 80, 40)
I0328 11:18:29.403243 140037153568640 efficientnet_model.py:390] Expand shape: (64, 80, 80, 240)
I0328 11:18:29.435106 140037153568640 efficientnet_model.py:393] DWConv shape: (64, 80, 80, 240)
I0328 11:18:29.459021 140037153568640 

### Testing - REVIEW IMSIZES IN TFRECORDS

In [5]:
# IF YOU CAME STRAIGHT HERE RUN THIS...

%cd /content/drive/MyDrive/training-effdet/automl/efficientdet
MODEL = 'efficientdet-d1'
if MODEL not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
    !tar xf {MODEL}.tar.gz

/content/drive/MyDrive/training-effdet/automl/efficientdet


In [ ]:
# This code box to test on default backup dir, else skip to next one

%cd /content/drive/MyDrive/training-effdet/automl/efficientdet

!python main.py --mode=eval \
      --model_name=efficientdet-d1 \
      --model_dir=/content/drive/MyDrive/training-effdet/backup_dir/backup  \
      --val_file_pattern=/content/drive/MyDrive/training-effdet/automl/efficientdet/tfrecord/PPU-6-train.tfrecord \
      --hparams=/content/drive/MyDrive/TFM/training-assets/PPU-6-config.yaml \
      --eval_batch_size=1 \
      --eval_samples=500 \

In [9]:
# Testing checkpoint on remote folder for test split
# 1. Take files away to some folder
# 2. Modify */checkpoint with the path of the model without extension in both rows
# 3. Run this and hope for the best

%cd /content/drive/MyDrive/training-effdet/automl/efficientdet

!python main.py --mode=eval \
      --model_name=efficientdet-d1 \
      --model_dir=/content/drive/MyDrive/ED-D1-backup_temp  \
      --val_file_pattern=/content/drive/MyDrive/TFM/training-assets/PPU-6-640-test.tfrecord \
      --hparams=/content/drive/MyDrive/TFM/training-assets/PPU-6-config.yaml \
      --eval_batch_size=1 \
      --eval_samples=520 \
      >> /content/logfile.txt

/content/drive/MyDrive/training-effdet/automl/efficientdet
2021-03-28 11:47:50.327442: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
I0328 11:47:52.518654 140124441188224 main.py:264] {'name': 'efficientdet-d1', 'act_type': 'swish', 'image_size': (640, 640), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 6, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': {1: 'backpack', 2: 'helmet', 3: 'drill', 4: 'extinguisher', 5: 'survivor', 6: 'rope'}, 'max_instances_per_image': 100, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch

In [10]:
!cp /content/logfile.txt /content/drive/MyDrive/TFM/effdet-d1-map.txt

## Inference on pictures

In [12]:
# copy all test images to a folder where no text files exist
import os

!mkdir /content/test-notxt/

piclist = [pic for pic in os.listdir('/content/raw-data/test') if pic[-1]!='t']
for idx, pic in enumerate(piclist):
    img = cv2.imread(f'/content/raw-data/test/{pic}') 
    img = cv2.resize(img, (imsize,imsize))
    cv2.imwrite(f'/content/test-notxt/{pic}', img)
    print(f'\r Resizing & copying {split} split... {idx+1} converted. Current size {img.shape}', end='')
 

 Resizing & copying test split... 520 converted. Current size (640, 640, 3)

In [13]:
# Inference of pictures
%cd /content/drive/MyDrive/training-effdet/automl/efficientdet



# First export a saved model.
saved_model_dir = '/content/savedmodel'
!rm -rf {saved_model_dir}

!python model_inspect.py --runmode=saved_model \
    --model_name=efficientdet-d1 \
    --ckpt_path=/content/drive/MyDrive/ED-D1-backup_temp \
    --saved_model_dir={saved_model_dir} \
    --hparams=/content/drive/MyDrive/TFM/training-assets/PPU-6-config.yaml 



# Then run saved_model_infer to do inference.
# Notably: batch_size, image_size must be the same as when it is exported.
serve_image_out = '/content/serve_image_out'
!mkdir {serve_image_out}

!python model_inspect.py --runmode=saved_model_infer \
    --saved_model_dir={saved_model_dir} \
    --model_name=efficientdet-d1 \
    --input_image='/content/test-notxt/*'  \
    --output_image_dir={serve_image_out} \
    --min_score_thresh=0  \
    --max_boxes_to_draw=10 \
    --hparams=/content/drive/MyDrive/TFM/training-assets/PPU-6-config.yaml 


/content/drive/MyDrive/training-effdet/automl/efficientdet
2021-03-28 11:52:34.652620: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-28 11:52:39.109644: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-28 11:52:39.110573: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-28 11:52:39.150377: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-28 11:52:39.151028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-03-28 11:52:39.1510

In [14]:
!zip -q -r /content/drive/MyDrive/TFM/effdet-d1-infer.zip /content/serve_image_out 

## remove stuff from colab once finished training

In [ ]:
# # # !rm -r /content/drive/MyDrive/training-effdet/automl
# # # !rm -r /content/drive/MyDrive/training-effdet/